<a href="https://colab.research.google.com/github/joaopaulolndev/deep-learning-keras-projects/blob/master/Games/RegressaoGames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Input
from keras.utils import np_utils
from keras.models import Model

In [0]:
#https://drive.google.com/open?id=1Q_ySHWpOEJxlZrB-ulldzQG8Eq-CxTc8

# Carrega arquivo de entrada 
file = drive.CreateFile({'id':'1Q_ySHWpOEJxlZrB-ulldzQG8Eq-CxTc8'}) 
file.GetContentFile('games.csv')  
base = pd.read_csv('games.csv')

In [0]:
base = base.drop('Other_Sales',axis = 1)
base = base.drop('Global_Sales',axis = 1)
base = base.drop('Developer',axis = 1)

In [0]:
base = base.dropna(axis=0)

In [0]:
base = base.loc[base['NA_Sales'] > 1]
base = base.loc[base['EU_Sales'] > 1]

In [0]:
nome_jogos = base.Name

In [0]:
base = base.drop('Name',axis = 1)

In [0]:
base['User_Score'] = base['User_Score'].astype(float)

In [0]:
base.dtypes

Platform            object
Year_of_Release    float64
Genre               object
Publisher           object
NA_Sales           float64
EU_Sales           float64
JP_Sales           float64
Critic_Score       float64
Critic_Count       float64
User_Score         float64
User_Count         float64
Rating              object
dtype: object

In [0]:
previsores = base.iloc[:, [0,1,2,3,7,8,9,10,11]].values

In [0]:
previsores 

array([['Wii', 2006.0, 'Sports', ..., 8.0, 322.0, 'E'],
       ['Wii', 2008.0, 'Racing', ..., 8.3, 709.0, 'E'],
       ['Wii', 2009.0, 'Sports', ..., 8.0, 192.0, 'E'],
       ...,
       ['Wii', 2007.0, 'Sports', ..., 8.0, 124.0, 'E10+'],
       ['PS2', 2001.0, 'Racing', ..., 7.9, 46.0, 'T'],
       ['PS2', 2003.0, 'Simulation', ..., 8.4, 42.0, 'T']], dtype=object)

In [0]:
venda_na = base.iloc[:,4].values
venda_eu = base.iloc[:,5].values
venda_jp = base.iloc[:,6].values

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [0]:
labelencoder = LabelEncoder()

In [0]:
previsores[:,0] = labelencoder.fit_transform( previsores[:,0] )
previsores[:,2] = labelencoder.fit_transform( previsores[:,2] )
previsores[:,3] = labelencoder.fit_transform( previsores[:,3] )
previsores[:,8] = labelencoder.fit_transform( previsores[:,8] )

In [0]:
# transformas as variaveis que eram categoricas e foram convertidas em numeros pelo label encoder agora converte em variaveis dummy
onehotencoder = OneHotEncoder(categorical_features=[0,2,3,8])
previsores = onehotencoder.fit_transform(previsores).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [0]:
camada_entrada = Input(shape=(61,))

camada_oculta1 = Dense( units=32, activation = 'sigmoid')(camada_entrada)
camada_oculta2 = Dense( units=32, activation = 'sigmoid')(camada_oculta1)

camada_saida1 = Dense( units=1, activation = 'linear')(camada_oculta2)
camada_saida2 = Dense( units=1, activation = 'linear')(camada_oculta2)
camada_saida3 = Dense( units=1, activation = 'linear')(camada_oculta2)

In [0]:
regressor = Model(inputs = camada_entrada, outputs = [camada_saida1,camada_saida2,camada_saida3])

In [0]:
regressor.compile(optimizer = 'adam', loss = 'mse')

In [0]:
regressor.fit(previsores, [venda_na, venda_eu, venda_jp], epochs = 5000, batch_size = 100)

Epoch 1/5000
258/258 [==============================] - 0s 2ms/step - loss: 45.8348 - dense_13_loss: 25.2919 - dense_14_loss: 18.9321 - dense_15_loss: 1.6108
Epoch 2/5000
258/258 [==============================] - 0s 42us/step - loss: 44.0637 - dense_13_loss: 24.5395 - dense_14_loss: 18.0213 - dense_15_loss: 1.5029
Epoch 3/5000
258/258 [==============================] - 0s 49us/step - loss: 42.4180 - dense_13_loss: 23.8295 - dense_14_loss: 17.1860 - dense_15_loss: 1.4025
Epoch 4/5000
258/258 [==============================] - 0s 36us/step - loss: 40.8998 - dense_13_loss: 23.1615 - dense_14_loss: 16.4130 - dense_15_loss: 1.3253
Epoch 5/5000
258/258 [==============================] - 0s 29us/step - loss: 39.3880 - dense_13_loss: 22.4749 - dense_14_loss: 15.6529 - dense_15_loss: 1.2602
Epoch 6/5000
258/258 [==============================] - 0s 33us/step - loss: 38.0514 - dense_13_loss: 21.8742 - dense_14_loss: 14.9695 - dense_15_loss: 1.2077
Epoch 7/5000
258/258 [=========================

In [0]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(previsores)